<a href="https://colab.research.google.com/github/jdasam/ant5015-2023/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제 1
- TODO로 표시된 함수 및 변수 선언 부분을 설명에 맞게 코드로 완성할 것
    - TODO이외의 부분을 수정하지 않아도 코드가 작동함
- 제출은 ``Assignment_1.py``파일에 완성한 함수를 복사 붙여넣기하여 제출 
  - 제출 전  `Assignment_1.py` 작동 확인
- 제출 전 메뉴바-런타임/다시 시작 및 모두 실행 했을 때 정상 작동하는 것을 꼭 확인
- 출제된 베이스 코드에 오타 및 에러 보고 시 가산점
- 제출 파일:
  1. ``Assignment_1_{학번}.py`` (예: ``Assignment_1_20240001.py``)
  2. ``shepard_tone_{학번}.wav`` (예: ``shepard_tone_20240001.wav``)
  - **파일 이름 형식이 잘못될 시 10점 감점**


In [ ]:
# Download .py file. You have to copy and paste the completed function to this py file and submit it.
!wget https://raw.githubusercontent.com/jdasam/ant5015-2023/main/Assignment_1.py

In [ ]:
import matplotlib.pyplot as plt
import IPython.display as ipd
import torch
from math import pi
import torchaudio
from matplotlib.colors import LogNorm

SR = 16000

## 문제 1: Pitch Scale 변환 및 음계 생성 (5점)
- Hz는 선형적인 스케일을 가진 주파수 단위
  - 일초에 같은 파형이 몇번 반복되는지를 나타냄
- MIDI Pitch는 가온 다(C4)를 60으로 하고 서양음계에서의 반음 차이를 1로 두는 단위
  - 서양음계에서 한 옥타브는 12개의 반음으로 이루어져있음
  - 한 옥타브 차이의 음은 MIDI Pitch상 12 만큼 차이남
  - 피아노 건반의 88개 음은 MIDI Pitch 21에서 시작해 108로 끝나는 정수에 해당함 
<img src=https://newt.phys.unsw.edu.au/jw/graphics/notesinvert.GIF>

- Hz를 Pitch로 변환하는 식 (평균율 기준)
  - $f(h) = \log_2(\frac{h}{440}) \times 12 + 69 $
  - h: Hz 
- MIDI Pitch를 Hz로 변환하는 식 (평균율 기준)
  - $f(\text{p})= 440 \times 2^{\frac{p-69}{12}}$
  - p: MIDI Pitch
  - f(p): MIDI Pitch에 해당하는 Hz
  - p=69일 때 f(p)=440Hz이며, p가 1늘어날 때마다 주파수가 $\times 2^{\frac{1}{12}}$ 비율로 증가함

In [ ]:
from math import log
def hz_to_midi_pitch(frequency):
  '''
  입력: 음높이의 Hz 단위 주파수 (float)

  출력: 입력 음높이를 MIDI Pitch Scale로 변환한 값 (float). 반올림할 필요 없음
  '''
  # TODO: 주어진 입출력 형식과 내용을 만족하는 함수를 완성하시오
  return 


def midi_pitch_to_hz(midi_pitch):
  '''
  입력: 음높이의 MIDI Pitch Scale (float)

  출력: 입력 음높이를 Hz로 변환한 값 (float)
  '''
  # TODO: 주어진 입출력 형식과 내용을 만족하는 함수를 완성하시오
  return 

for harmonic_index in range(1,10):
  test_freq = 440 * harmonic_index
  print(f'{test_freq} Hz is {hz_to_midi_pitch(test_freq):.2f} in MIDI Pitch')

for midi_pitch in range(60, 73):
  print(f'{midi_pitch} in MIDI Pitch is {midi_pitch_to_hz(midi_pitch):.2f} Hz')


## 문제 2-1: 코드 및 음계 합성하기 (20점)
- 주어진 `make_sine_wave()` 및 문제1번에서 완성한 함수를 활용하여 아래 함수들을 완성하시오
  - `make_sine_wave()` 함수가 어떻게 작동하는지 테스트

In [ ]:
def make_sine_wave(freq, amp, dur, sr):
  '''
  Inputs:
    freq: sine wave의 주파수 (Hz)
    amp: sine wave의 진폭 (0~1)
    dur: sine wave의 지속시간 (sec)
    sr: sampling rate (Hz)
  Output:
    sine wave (torch.Tensor), shape: (num_samples,)
  '''
  num_samples = dur * sr
  time_frame = torch.arange(num_samples)
  time_frame_sec = time_frame / sr
  return amp * torch.sin(2 * pi * freq * time_frame_sec)


example_sine_wav = make_sine_wave(440, 1, 1, SR)
plt.plot(example_sine_wav[:500])
ipd.display(ipd.Audio(example_sine_wav, rate=SR))

- TODO: `generate_multi_pitch_tone()`을 완성하여 다음 코드를 실행하여 C장조 코드 (도,미,솔)이 생성되는지 확인한다

In [ ]:
def generate_multi_pitch_tone(pitch_list, amp_list, duration, sr):
  '''
  입력: pitch_list: List of length n that contains n number of MIDI Pitch 
                   n 개의 MIDI Pitch 정수를 포함하고 있는 리스트 (n>=2)
       amp_list : List of length n that contains amplitude for each pitch   (0<amp<=1)
                   pitch_list에 있는 n개의 MIDI Pitch 각각에 해당하는 amplitude 
                   (각각의 amplitude는0보다 크고 1보다 작거나 같음)
      duration: 출력되는 소리의 길이 (float)
      sr: sampling rate

  출력: 입력으로 주어진 리스트 속에 있는 모든 pitch가 포함된 1채널 오디오 샘플 (sample_rate=sr, torch.Tensor 형태, 1D tensor)
       별도의 채널 디멘션 없이 1차원 텐서로 출력
        예) 입력:  
          pitch_list = [60, 64, 67]
          amp_list = [1, 0.7, 0.5]
          출력: MIDI Pitch 60이며 amplitude가 1인 소리, Pitch 64이며 amp 0.7인 소리, 
               Pitch 67이며 amp 0.5인 소리 세 개가 duration만큼 동시에 재생되는 오디오 샘플 
    
  '''
  assert len(pitch_list) == len(amp_list)
  # TODO: 주어진 입출력 형식과 내용을 만족하는 함수를 완성하시오

  return

c_major_chord = [60, 64, 67]
amp_list = [1, 1, 1]
chord_tone = generate_multi_pitch_tone(c_major_chord, amp_list, 3, sr=SR)
assert chord_tone.ndim == 1, "Output should be 1D tensor"
ipd.Audio(chord_tone, rate=SR)

In [ ]:
def make_major_scale_hz_sequence(midi_pitch):
  '''
  입력: 음높이의 MIDI Pitch Scale (float)

  출력 (list of float): 입력된 MIDI Pitch를 1음(Do)으로 하는 장음계 8개의 음의 Hz
  Hint: 장음계는 3음과 4음 (Mi, Fa) 사이가 반음, 7음과 8음 (Si, Do) 사이가 반음이며 나머지는 모두 온음 간격이다.

  예: 입력이 60 (C4)라면, C4, D4, E4, F4, G4, A4, B4, C5에 해당하는 음고 Hz를 list로 출력한다. 
  '''
  # TODO: 주어진 입출력 형식과 내용을 만족하는 함수를 완성하시오
  return


def generate_sequence_of_pitch(alist_of_hz, duration, sr):
  '''
  입력: 
    alist_of_hz (list): A list of float (Hz)
    duration (float): Duration of each frequency (seconds)

  출력: 입력으로 주어진 리스트 속에 있는 모든 Hz가 순서대로 등장하는 오디오 샘플 (sample_rate=sr, torch.Tensor 형태)

  예) 입력:  
          alist_of_hz = [440, 660, 880]
          duration = 1
          출력: 440 Hz 사인파가 1초 동안 재생된 뒤 660 Hz 사인파가 1초 동안 재생된 뒤 880 Hz 사인파가 1초 동안 재생되는 오디오 샘플
  '''
  # TODO: 주어진 입출력 형식과 내용을 만족하는 함수를 완성하시오
  return

fund_note = 69 # MIDI pitch for A4
major_scale_in_hz = make_major_scale_hz_sequence(fund_note)
scale_sine = generate_sequence_of_pitch(major_scale_in_hz, duration=0.5, sr=SR)

ipd.display(ipd.Audio(scale_sine, rate=SR))

## 문제 2-2: 순정률 (Just Intonation) vs 평균율 (Equal Temperament) (5점)
- 음고가 2배가 되면 같은 음으로 인지되는 것을 옥타브라고 한다.
  - 한 옥타브 안에 있는 음표들 끼리 어떤 음고 비율을 가지게 만들까?
- [순정률](https://ko.wikipedia.org/wiki/%EC%88%9C%EC%A0%95%EB%A5%A0)은 각각의 음고 (Hz) 관계가 정수배로 표현되는 조율법 의미한다.
  - 정수배 비율로 표현된다는 것은 유리수, 즉 분모와 분자가 정수인 분수로 표현할 수 있다는 말이다.
  - 순정 8도는 근음과 높은음의 음고 비율이 1:2가 되는 관계다.
  - 순정 5도는 근음과 높은음의 음고 비율이 2:3이 되는 관계다
  - 순정 장3도는 근음과 높은음의 음고 비율이 4:5가 되는 관계다.
  - 순정 단3도는 근음과 높은음의 음고 비율이 5:6이 되는 관계다.
- 평균율은 한 옥타브를 12개의 반음으로 쪼개고 각각의 반음 간의 음고 비율이 모두 일정하게 만든다.
  - 반음 끼리의 주파수 비율은 $2^{\frac{1}{12}}$
  - 화음을 만들때 음끼리 정확한 정수배가 되지 않는다

In [ ]:
def make_just_intonation_triad_in_midi_pitch(fundamental_pitch):
  '''
  This function takes a fundamental_pitch in MIDI pitch then returns a list of three MIDI pitch
  that corresponds to major triad for a given fundamental frequency

  fundamental_pitch: 근음의 음고 MIDI Pitch
  output= [근음의 MIDI_pitch, 근음 순정 장3도의 MIDI_Pitch, 근음기준 순정 5도의 MIDI Pitch]
  '''

  # TODO: 주어진 입출력 형식과 내용을 만족하는 함수를 완성하시오

  return


'''
다음 예제는 순정율로 만든 장화음 소리와 평균율로 만들어진 장화음 소리를 비교하는 것이다.
순정율 함수를 정확하게 구현하였다면 평균율에 비해서 맥놀이 없이 안정적인 소리가 재생된다. 
'''
fund_pitch = 60
just_intonation_chord = make_just_intonation_triad_in_midi_pitch(fund_pitch)
amp_list = [1, 1, 1]
just_chord_tone = generate_multi_pitch_tone(just_intonation_chord, amp_list, 3, sr=SR)
print(f'Below is the Chord Tone with Just Intonation ({just_intonation_chord:})')
ipd.display(ipd.Audio(just_chord_tone, rate=SR))

equal_temperament_chord = [fund_pitch, fund_pitch+4, fund_pitch+7]
equal_chord_tone = generate_multi_pitch_tone(equal_temperament_chord, amp_list, 3, sr=SR)
print(f'Below is the Chord Tone with Equal Temperament  ({equal_temperament_chord}). You can hear beat(맥놀이).')
ipd.display(ipd.Audio(equal_chord_tone, rate=SR))

## 문제3: 셰퍼드 톤 합성하기 (20점)
- [셰퍼드 톤 설명 영상](https://youtu.be/LVWTQcZbLgY)
- [Wikipeda](https://en.wikipedia.org/wiki/Shepard_tone)
- 사람이 소리의 높이와 세기를 어떻게 인지하는지를 고려하여 완성한다.

In [ ]:
class ShepardToneGenerator:
  def __init__(self, sr, dur_per_step, num_iteration):
    self.sr = sr #sampling rate
    self.dur = dur_per_step # 한 음 당 길이 (초) 
    self.iter = num_iteration # 전체 시퀀스를 몇번 반복할 것인지
    self.fadeout_margin = self.dur*0.1 # 음 끝의 페이드아웃 길이
    
    self.pitch_combinations = self.define_pitch()
    self.amplitude_combinations = self.define_amplitude()
  
  def define_pitch(self):
    '''
    입력: 없음

    출력: List of List of MIDI pitch의 Tensor형태 
         MIDI Pitch 리스트들의 리스트
        (예: [ [60, 64, 67], [61, 65, 68], [62, 66, 69] ])
    '''
    output = torch.zeros([24, 3]) # 24 note combinations with 3 notes each
    # TODO: 셰퍼드 톤을 만들기 위한 output을 완성하시오
    return output

  def define_amplitude(self):
    '''
    입력: 없음

    출력: List of List of amplitude의 Tensor형태 
         self.pitch_combinations의 각각에 해당하는 소리의 amplitude
        (예: [ [1.0, 0.2, 0.8], [1.0, 0.5, 0.5], [1.0, 0.8, 0.2] ])

    힌트: 사람이 듣는 소리의 크기는 amplitude의 log값에 비례하므로 amplitude를 이에 맞게 조절해야한다
    '''
    output = torch.zeros([24,3]) # 24 note combinations with 3 notes each
    # TODO: 셰퍼드 톤을 만들기 위한 output을 완성하시오
    return output

  def generate(self):
    '''
    You don't need to modify this function
    '''
    outputs = []
    for i in range(self.iter):
      for j in range(len(self.pitch_combinations)):
        current_pitches = self.pitch_combinations[j]
        current_amplitudes = self.amplitude_combinations[j]
        audio = generate_multi_pitch_tone(current_pitches, current_amplitudes, self.dur, self.sr)
        audio[-int(self.fadeout_margin*self.sr):] *= torch.logspace(0, -3, int(self.fadeout_margin*self.sr))
        outputs.append(audio)
    return torch.cat(outputs, dim=-1)


shetone = ShepardToneGenerator(SR,0.3,3)
shepard_audio = shetone.generate()
ipd.Audio(shepard_audio, rate=SR)

## 문제4: Spectrogram Plot
- Spectrogram은 Short-time Fourier Transform을 이용하여 오디오 샘플을 시간에 따른 주파수별 에너지 변화로 표현한 2D 매트릭스이다.
- 문제 4번에서는 위에서 생성한 `shepard_audio`를 사용해도 되며 아래 코드를 실행해서 다른 오디오를 불러와도 괜찮다.
- 주어진 클래스 ``SpectrogramConverter``의 method를 완성하는 것이 목표


In [ ]:
class SpectrogramConverter:
  def __init__(self, n_fft, hop_size, sample_rate, num_mels, power=2.0):
    self.n_fft = n_fft
    self.hop_size = hop_size
    self.sr = sample_rate
    self.num_mels = num_mels
    self.window_tensor = torch.hann_window(n_fft)
    self.power = power

    self.mel_fb = self.get_melfilterbank()

  def get_spectrogram(self, audio):
    '''
    audio: 입력 오디오 샘플(torch.Tensor).
    n_fft: fft size. 한 번에 몇 개의 오디오 샘플을 푸리에 변환할지 결정
    hop_size: 오디오 

    출력: 오디오에 대해 fft_size = n_fft를 사용한 스펙트로그램 (torch.Tensor)
        absolute value만을 사용
    '''
    if audio.ndim == 2:
      if audio.shape[1] > audio.shape[0]:
        audio = audio.mean(dim=0)
      else:
        audio = audio.mean(dim=1)
    assert audio.ndim == 1, "Audio must be 1D tensor"
    # 문제 4-1
    # TODO: Complete this function with only using torch.fft.fft() function
    # You HAVE to use self.window_tensor, self.n_fft, self.hop_size, self.power, and audio
    # You have to use torch.fft.fft() and torch.abs()
    # Usually, we use SQUARED VALUE (self.power=2.0) of the absolute value of the fft result as the spectrogram
    # You have to use torch.stack() to concatenate the result


    return
  


  def get_melfilterbank(self):
    '''
    입력: 없음

    출력: mel filterbank (torch.Tensor)
    '''
    mel_fb = torchaudio.transforms.MelScale(n_mels=self.num_mels, sample_rate=self.sr, f_min=0.0, f_max=self.sr/2, n_stft=self.n_fft//2+1).fb
    return mel_fb
  
  def frequency_bin_to_hz(self, bin_index):
    '''
    입력: bin_index: Spectrogram에서 frequency bin의 인덱스

    출력: sr,n_fft에 해당하는 스펙트로그램 frequency bin index의 frequency Hz
    '''
    # 문제 4-2
    # TODO: 함수를 완성하시오
    # Hint: use self.sr and self.n_fft
    return 

  def time_bin_to_second(self, bin_index):
    '''
    입력: bin_index: Spectrogram에서 time bin의 인덱스
        sr: sampling rate (Hz)
        n_fft: fft size (sample)
        hop_size: Spectrogram의 각 프레임이 입력 오디오 상에서 몇 오디오 샘플 씩 떨어져있는지를 나타냄 

    출력: sr,n_fft,hop_size에 해당하는 스펙트로그램 time bin index가 입력 오디오 몇 초에 해당하는지를 반환
      second (float): 스펙트로그램 time frame의 frame_index에 해당하는 오디오 샘플의 시간 위치 (초). Window가 시작되는 샘플을 기준으로 함
    '''
    # 문제 4-2
    # TODO: 함수를 완성하시오
    # Hint: use self.sr, self.n_fft, and self.hop_size

    return

  def hz_to_mel(self, hz):
    '''
    입력: hz: frequency (torch.Tensor)

    출력: hz에 해당하는 mel scale의 값 (torch.Tensor)

    입력과 출력의 shape은 동일함
    '''
    # 문제 4-3
    # TODO: 함수를 완성하시오
    return 
  

  def convert_spec_to_mel_spec(self, spec):
    '''
    입력: spec: spectrogram (torch.Tensor)

    출력: mel spectrogram (torch.Tensor)
    '''
    # 문제 4-3
    # TODO: torch.mm(), self.mel_fb, spec을 이용하여 함수를 완성하시오
    # Hint: Mel Filterbank는 spectrogram 각각의 frequency bin에 대해 각각의 mel scale에 대한 weight 값을 표현한 것임
    # Hint: self.mel_fb는 ( n_fft//2+1, num_mels)의 shape을 가짐. 
    # Hint: Output은 (num_mels, time_frame)의 shape을 가짐 
    return

  def amplitude_to_db(self, spectrogram, eps=1e-10):
    '''
    spectrogram: 입력 스펙트로그램 (torch.Tensor)

    출력: 입력 스펙트로그램을 dB scale로 변환한 결과 (torch.Tensor)
    '''
    # 문제4-4
    # TODO: Complete this function
    # You have to use torch.log10() and torch.clamp()
    # To avoid log(0), you have to use eps to clamp the spectrogram
    # Hint: The spectrogram is already in POWER (SQUARED VALUE)

    return
  
  def __call__(self, audio):
    '''
    audio: 입력 오디오 샘플(torch.Tensor).
    
    output: mel spectrogram in dB scale (torch.Tensor)
    '''
    # 문제 4-5
    # TODO: Complete this function using self.get_spectrogram(), self.convert_spec_to_mel_spec(), and self.amplitude_to_db()

    return 
  
spec_converter = SpectrogramConverter(n_fft=1024, hop_size=512, num_mels=128, sample_rate=16000)

In [ ]:
# Option 1
your_audio = shepard_audio

# Option 2
# your_audio, sr = torchaudio.load("your_audiofile.wav")

### 문제 4-1: Spectrogram 생성 (15점)
- `SpectrogramConverter.get_spectrogram()`을 완성하시오.
- 주파수가 Linear Scale인 Spectrogram을 생성한다 
  - 주파수가 Linear Scale이라는 것은 주파수를 나타내는 y축에 표현된 등간격이 100, 200, 300, 400 처럼 등차수열로 커지는 것을 의미한다.
  - 주파수가 Log Scale로 표현되면 y축에 표현된 등간격이 100, 1000, 10000, 100000 처럼 등비수열로 커지는 것을 의미한다.

![image](https://i.stack.imgur.com/Jg5EG.png)
이 그림에서 K는 fft size에 해당하며 Q는 hop size에 해당한다.

In [ ]:
spec_converter = SpectrogramConverter(n_fft=1024, hop_size=512, num_mels=128, sample_rate=16000)
spec = spec_converter.get_spectrogram(your_audio)

ref_spec = torchaudio.transforms.Spectrogram(n_fft=1024, hop_length=512, center=False)(your_audio)
plt.imshow(spec, origin='lower', aspect='auto', interpolation='nearest', norm=LogNorm())

assert torch.allclose(spec, ref_spec), "Your output spectrogram is different with torchaudio.Spectrogram"

### 문제 4-2: Spectrogram 축의 값 변환 (12점)
-  `SpectrogramConverter.frequency_bin_to_hz()` 와 `SpectrogramConverter.time_bin_to_second()`를 완성하시오
- 위에서 그려진 그래프의 x축 y축의 레이블 값은 각각 Frame / Bin (칸) 인덱스를 의미한다.
  - `spec.shape`이 `torch.size([513, 1801])`이라면 이는 주파수 축으로 513개의 칸이 있고 시간 축으로는 1801개의 칸이 있다는 것을 의미한다. 
  - 예: x축 값 250은 스펙트로그램에서 250번째 시간축 칸을 의미한다. 
  - 예: y축 값 200은 200번째 주파수 칸을 의미한다.

- 시간 축과 주파수 축을 각각 초(second)와 Hz 단위로 변환하면 스펙트로그램의 데이터를 더 잘 이해할 수 있다.
- Spectrogram을 생성할 때 사용하였던 파라미터(n_fft, hop_size 등)를 이용하여 스펙트로그램 각각의 칸이 어떤 값을 의미하는지 변환할 수 있다.

- **Hint:** 함수가 올바르게 완성되었다면 아래 plot 코드를 실행했을 때 x축은 초 단위, y축은 Hz 단위로 나오게 된다.
  - 실제 오디오의 길이가 그래프 x축 가장 오른쪽 틱 레이블과 비슷한지 확인한다.
  - 그래프 y축 가장 위쪽 틱 레이블이 Sampling rate의 절반과 비슷한지 확인한다.
    - Nyquist's Theorem에 따라 샘플링 레이트 절반을 넘는 주파수는 관측할 수 없다.
    - 즉 STFT 결과에서 가장 높은 주파수 칸은 샘플링레이트 절반에 해당하는 주파수이다.

In [ ]:
def change_ytick_to_frequency(spec_converter):
  prev_yticks = plt.yticks()[0][1:-1] 
  ytick_labels= [spec_converter.frequency_bin_to_hz(bin_index) for bin_index in prev_yticks]
  plt.yticks(ticks=prev_yticks, labels=ytick_labels)

def change_xtick_to_seconds(spec_converter):
  prev_xticks = plt.xticks()[0][1:-1]
  xtick_labels= [spec_converter.time_bin_to_second(bin_index) for bin_index in prev_xticks]
  plt.xticks(ticks=prev_xticks, labels=xtick_labels)


fft_size=1024
hop_size=512
spec_converter = SpectrogramConverter(n_fft=1024, hop_size=512, num_mels=128, sample_rate=16000)
spec = spec_converter.get_spectrogram(your_audio)

plt.figure(figsize=(10,10))
plt.imshow(spec, origin='lower', aspect='auto', interpolation='nearest', norm=LogNorm())
change_ytick_to_frequency(spec_converter)
change_xtick_to_seconds(spec_converter)
plt.ylabel('Frequency (Hz)') 
plt.xlabel('Time (seconds)') 

### 문제 4-3 Mel Scale 생성 후 Mel Spectrogram 만들기 (10점)
- `SpecConverter.hz_to_mel()` 완성시키기
  - [Mel Scale](https://en.wikipedia.org/wiki/Mel_scale) 참조
- `SpecConverter.convert_spec_to_mel_spec()` 완성시키기 

In [ ]:
spec_converter = SpectrogramConverter(n_fft=1024, hop_size=512, num_mels=128, sample_rate=16000)
mel_scale = spec_converter.get_melfilterbank()

hz_linspace = torch.linspace(0, 8000, 100)

# TODO: Complete SpectrogramConverter.hz_to_mel() and use it
mel_linspace = spec_converter.hz_to_mel(hz_linspace)

plt.figure(figsize=(10,15))
plt.subplot(2,1,1)
plt.imshow(mel_scale, aspect='auto', origin='lower', interpolation='nearest')
change_ytick_to_frequency(spec_converter)
plt.ylabel('Frequency (Hz)') 
plt.xlabel('Mel Filterbank Index') 

plt.subplot(2,1,2)
plt.plot(mel_linspace, hz_linspace)

In [ ]:
# TODO: Complete SpectrogramConverter.convert_spec_to_mel_spec
mel_spec= spec_converter.convert_spec_to_mel_spec(spec)
ref_spec = torchaudio.transforms.MelScale(sample_rate=16000, n_mels=128, n_stft=513)(spec)

assert torch.allclose(mel_spec, ref_spec), "Your output mel spectrogram is different with using torchaudio.MelScale"

# Plot
plt.figure(figsize=(10,10))
plt.title("Mel Spectrogram")
plt.ylabel("Mel Bin")
plt.imshow(mel_spec, origin='lower', aspect='auto', interpolation='nearest', norm=LogNorm()) 
change_xtick_to_seconds(spec_converter)
plt.xlabel('Time (seconds)') 

### 문제 4-4 Amplitude to dB (8점)
- Amplitude Scale의 Spectrogram을 dB scale로 변경
- x축 y축의 스케일을 변경하는 것이 아니라 (x,y) 칸에 들어있는 모든 숫자들의 스케일을 dB로 변경

In [ ]:
# TODO: You have to complete SpectrogramConverter.amplitude_to_db
db_spec = spec_converter.amplitude_to_db(mel_spec)
ref_spec = torchaudio.transforms.AmplitudeToDB()(mel_spec)

assert torch.allclose(db_spec, ref_spec), "Your output spectrogram is different with using torchaudio.AmplitudeToDB"

plt.figure(figsize=(15,10))
plt.subplot(2,2,1)
plt.title("Amplitude Mel Spectrogram")
plt.imshow(mel_spec, origin='lower', aspect='auto', interpolation="nearest")
plt.colorbar()
change_xtick_to_seconds(spec_converter)
plt.ylabel('Mel Bin')
plt.subplot(2,2,2)
plt.title("Amplitude Mel Spectrum (Single frame of spectrogram)")
plt.plot(mel_spec[:,mel_spec.shape[1]//2] )
plt.xlabel('Mel Bin')
plt.subplot(2,2,3)
plt.title("dB Mel Spectrogram")
plt.ylabel('Mel Bin')
plt.imshow(db_spec, origin='lower', aspect='auto', interpolation="nearest")
change_xtick_to_seconds(spec_converter)
plt.colorbar()
plt.subplot(2,2,4)
plt.title("dB Mel Spectrum (Single frame of spectrogram)")
plt.plot(db_spec[:,db_spec.shape[1]//2] )
plt.xlabel('Mel Bin')

### 문제 4-5: Audio to MelSpectrogram 완성 (5점)
  - ``SpectrogramConverter``의 ``__call__()`` 완성


In [ ]:
spec_converter = SpectrogramConverter(n_fft=1024, hop_size=512, num_mels=128, sample_rate=16000)

mel_db_spec = spec_converter(your_audio)

ref_spec = torchaudio.transforms.AmplitudeToDB()(torchaudio.transforms.MelSpectrogram(n_fft=1024, hop_length=512, center=False)(your_audio))

plt.imshow(mel_db_spec, origin='lower', aspect='auto', interpolation='nearest')
assert torch.allclose(mel_db_spec, ref_spec), "Your output spectrogram is different with torchaudio.MelSpectrogram"

## Assignment_1.py 점검
- 이 노트북에서 완성한 함수를 `Assignment_1.py`에 모두 복사 붙여넣고, wav 파일이나 png 플롯들이 모두 정상적으로 출력되는지 확인
- 파일 이름을 `Assignment_1_{학번}.py`으로 바꾸고, `shepard_tone_{학번}.wav`와 함께 사이버 캠퍼스에 제출


In [ ]:
!python3 Assignment_1.py